In [ ]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
# import slippi as slp=
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
# import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags
import matplotlib.pyplot as plt

In [ ]:
full_df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\mango_all_game_data_df.parquet")
print(full_df.columns)

<h2> Restrict to Interesting Columns </h2>

In [ ]:
df = full_df[['num_players','is_teams','player_1_character_name','player_2_character_name','player_1_type_name','player_2_type_name',
              'player_1_netplay_code', 'player_1_netplay_name','player_2_netplay_code', 'player_2_netplay_name',
              'stage_name', 'is_frozen_ps','conclusive','winning_player','player_1_win', 'player_2_win',
              'date', 'duration',]]

<h2> Count Unique Values </h2>
Conclusions:

- There are few games with more than two players

- There are few team games

- Battlefield is the most common stage, almost no games were played on the non-competitive maps.

- No True values for is_frozen_ps

- About one quarter of the games had no winner

- There are some CPU games, but relatively few

In [ ]:
cols = ['num_players','is_teams', 'stage_name', 'is_frozen_ps', 'conclusive','winning_player','player_1_type_name','player_2_type_name']
for col in cols:
    print(df[col].value_counts())
    print()

<h2> Filter Games </h2>
Filter the games to the ones that are interesting and count some more interesting values.

In [ ]:
df_r = df.loc[df['num_players'] == 2]
df_r = df_r.loc[df_r['is_teams'] == False]
df_r = df_r.loc[df_r['conclusive'] == True]
df_r = df_r.loc[df_r['player_1_type_name'] == 'HUMAN']
df_r = df_r.loc[df_r['player_2_type_name'] == 'HUMAN']
# Define the list of stage names to include
stages_to_include = [
    'BATTLEFIELD',
    'YOSHIS_STORY',
    'DREAM_LAND_N64',
    'POKEMON_STADIUM',
    'FINAL_DESTINATION',
    'FOUNTAIN_OF_DREAMS'
]

# Filter the dataframe for the desired stages
df_r = df_r[df_r['stage_name'].isin(stages_to_include)]

# Filter for rows where at least one of the player's netplay code is 'MANG#0'
df_r = df_r[(df_r['player_1_netplay_code'] == 'MANG#0') | (df_r['player_2_netplay_code'] == 'MANG#0')]

# Display the number of games left
print('Number of games: ', df_r.shape[0],'\n')

# Count the number of games on each stage
print(df_r['stage_name'].value_counts(), '\n')

# Concatenate the two columns into one Series
all_netplay_codes = pd.concat([df_r['player_1_netplay_code'], df_r['player_2_netplay_code']])

# Count the occurrences of each netplay code
code_counts = all_netplay_codes.value_counts()

# Display the top 10 most common (not MANG#0) values
top_10 = code_counts[1:]
print('Most common opponents')
print(top_10)


<h2> Create Mango Focused Dataframe </h2>

In [ ]:
# Filter to include only games where MANG#0 is playing
df_mango = df_r[(df_r['player_1_netplay_code'] == 'MANG#0') | (df_r['player_2_netplay_code'] == 'MANG#0')]

# Create the new dataframe
df_mango_stats = df_mango.apply(
    lambda row: pd.Series({
        'mango_character_name': row['player_1_character_name'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_character_name'],
        'opponent_netplay_code': row['player_1_netplay_code'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_netplay_code'],
        'opponent_netplay_name': row['player_1_netplay_name'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_netplay_name'],
        'opponent_character_name': row['player_1_character_name'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_character_name'],
        'stage_name': row['stage_name'],
        'mango_win': row['player_1_win'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_win'],
        'date': row['date'],
        'duration': row['duration']
    }), axis=1
)
# Normalize the opponent_netplay_code
df_mango_stats['normalized_netplay_codes'] = df_mango_stats['opponent_netplay_code'].apply(lambda x: x.split('#')[0])
df_mango_stats.head()

In [ ]:
print('Mango played agaist', len(df_mango_stats['opponent_netplay_code'].unique()), 'unique netplay codes.')
print('Mango played agaist', len(df_mango_stats['normalized_netplay_codes'].unique()), 'netplay code prefexes.\n')
print(df_mango_stats['opponent_netplay_code'].value_counts()[:20])


In [ ]:
print(df_mango_stats['normalized_netplay_codes'].value_counts()[:20])


LUCK, IBDW seem to have has more than one netplay code.

To Do: Display main(s) of each opponent and how many games mango has played against their main as fox and falco.

In [ ]:

print('Mango\'s 10 most commonly played characters:')
print(df_mango_stats['mango_character_name'].value_counts()[:10])

<h2> Some Interesting Charts </h2>
Conclusions:

- Mango prefers Falco over Fox except against Yoshi and Puff.

- Mango vastly prefers Falco over Fox agaist Fox.

- Mango plays Batlefield the most.

In [ ]:
falco_games = df_mango_stats[df_mango_stats['mango_character_name'] == 'FALCO']
fox_games = df_mango_stats[df_mango_stats['mango_character_name'] == 'FOX']


In [ ]:


# Count the number of games played against each character as Fox and Falco
fox_character_counts = fox_games['opponent_character_name'].value_counts().reset_index()
falco_character_counts = falco_games['opponent_character_name'].value_counts().reset_index()

# Rename columns for clarity
fox_character_counts.columns = ['character', 'fox_count']
falco_character_counts.columns = ['character', 'falco_count']

# Merge the two dataframes on the character column
merged_counts = pd.merge(fox_character_counts, falco_character_counts, on='character', how='outer').fillna(0)

# Sort by the total count for better visualization
merged_counts['total_count'] = merged_counts['fox_count'] + merged_counts['falco_count']
merged_counts = merged_counts.sort_values('total_count', ascending=False)

plt.figure(figsize=(15, 8))
index = range(len(merged_counts['character']))  # X-axis position for each character
bar_width = 0.35

# Plot bars for Fox and Falco
fox_bars = plt.bar(index, merged_counts['fox_count'], bar_width, label='Fox')
falco_bars = plt.bar([i + bar_width for i in index], merged_counts['falco_count'], bar_width, label='Falco')

plt.xlabel('Character')
plt.ylabel('Number of Games')
plt.title('Number of Games Mango Played Against Each Character as Fox and Falco')
plt.xticks([i + bar_width / 2 for i in index], merged_counts['character'], rotation=90)
plt.legend()

# Add text annotations above the bars
for bar in fox_bars + falco_bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()



In [ ]:
characters = ['MARTH', 'FOX', 'FALCO', 'CAPTAIN_FALCON', 'SHEIK', 'JIGGLYPUFF']
stages = ['BATTLEFIELD', 'YOSHIS_STORY', 'POKEMON_STADIUM', 'DREAM_LAND_N64', 'FOUNTAIN_OF_DREAMS', 'FINAL_DESTINATION']

# Colors for Fox and Falco bars
fox_color = 'blue'
falco_color = 'red'

# Loop through each character to create a histogram
for character in characters:
    plt.figure(figsize=(12, 6))
    for stage in stages:
        # Count games where Mango played as Fox against this character on this stage
        fox_count = fox_games[(fox_games['opponent_character_name'] == character) & (fox_games['stage_name'] == stage)].shape[0]
        # Count games where Mango played as Falco against this character on this stage
        falco_count = falco_games[(falco_games['opponent_character_name'] == character) & (falco_games['stage_name'] == stage)].shape[0]
        
        # Bar positions for this stage
        index = stages.index(stage)
        fox_bar = plt.bar(index - 0.2, fox_count, 0.4, label='Fox' if stage == stages[0] else "", color=fox_color)
        falco_bar = plt.bar(index + 0.2, falco_count, 0.4, label='Falco' if stage == stages[0] else "", color=falco_color)

        # Annotate bars with the number of games
        plt.text(index - 0.2, fox_count, str(fox_count), ha='center', va='bottom')
        plt.text(index + 0.2, falco_count, str(falco_count), ha='center', va='bottom')
    
    plt.xlabel('Stage')
    plt.ylabel('Number of Games')
    plt.title(f'Number of Games Mango Played as Fox and Falco Against {character}')
    plt.xticks(range(len(stages)), stages, rotation=45)

    if character == characters[0]:
        plt.legend()

    plt.tight_layout()
    plt.show()



In [ ]:

# Filter games where Mango played as FALCO and opponent as FOX
falco_vs_fox_games = falco_games[falco_games['opponent_character_name'] == 'FOX']

# Find the top 5 opponent_netplay_code
top_opponents = falco_vs_fox_games['opponent_netplay_code'].value_counts().head(10).index

# Prepare the data for plotting
stage_counts = pd.DataFrame(index=top_opponents, columns=stages)

# Populate the DataFrame with the number of games for each opponent and stage
for opponent in top_opponents:
    for stage in stages:
        count = falco_vs_fox_games[(falco_vs_fox_games['opponent_netplay_code'] == opponent) & (falco_vs_fox_games['stage_name'] == stage)].shape[0]
        stage_counts.loc[opponent, stage] = count

# Set consistent colors for each stage
colors = plt.cm.viridis(np.linspace(0, 1, len(stages)))

# Plotting
stage_counts.plot(kind='bar', figsize=(15, 8), width=0.8)
plt.xlabel('Opponent Netplay Code')
plt.ylabel('Number of Games')
plt.title('Number of Games Against Top 10 FOX Players on Each Stage (Mango as FALCO)')
plt.legend(title='Stage Name')

# Set consistent colors for each stage
colors = plt.cm.viridis(np.linspace(0, 1, len(stages)))  # Generates a colormap
for i, stage in enumerate(stages):
    plt.gca().get_legend()

plt.show()


In [ ]:
# Filter games where Mango played as FALCO and opponent as FOX
falco_vs_fox_games = falco_games[falco_games['opponent_character_name'] == 'FOX']

# Normalize the opponent_netplay_code
falco_vs_fox_games['normalized_opponent'] = falco_vs_fox_games['opponent_netplay_code'].apply(lambda x: x.split('#')[0])

# Find the top 5 normalized opponent_netplay_code
top_opponents = falco_vs_fox_games['normalized_opponent'].value_counts().head(10).index

# Prepare the data for plotting
stage_counts = pd.DataFrame(index=top_opponents, columns=stages)

# Populate the DataFrame with the number of games for each normalized opponent and stage
for opponent in top_opponents:
    for stage in stages:
        count = falco_vs_fox_games[
            (falco_vs_fox_games['normalized_opponent'] == opponent) & 
            (falco_vs_fox_games['stage_name'] == stage)
        ].shape[0]
        stage_counts.loc[opponent, stage] = count

# Plotting
stage_counts.plot(kind='bar', figsize=(15, 8), width=0.8, color=plt.cm.viridis(np.linspace(0, 1, len(stages))))
plt.xlabel('Opponent Netplay Code')
plt.ylabel('Number of Games')
plt.title('Number of Games Against Top 10 FOX Players on Each Stage (Mango as FALCO)')
plt.legend(title='Stage Name', labels=stages)

plt.show()


In [ ]:
# Filter games where Mango played as FALCO and opponent as FOX
falco_vs_fox_games = falco_games[falco_games['opponent_character_name'] == 'MARTH']

# Normalize the opponent_netplay_code
falco_vs_fox_games['normalized_opponent'] = falco_vs_fox_games['opponent_netplay_code'].apply(lambda x: x.split('#')[0])

# Find the top 5 normalized opponent_netplay_code
top_opponents = falco_vs_fox_games['normalized_opponent'].value_counts().head(10).index

# Prepare the data for plotting
stage_counts = pd.DataFrame(index=top_opponents, columns=stages)

# Populate the DataFrame with the number of games for each normalized opponent and stage
for opponent in top_opponents:
    for stage in stages:
        count = falco_vs_fox_games[
            (falco_vs_fox_games['normalized_opponent'] == opponent) & 
            (falco_vs_fox_games['stage_name'] == stage)
        ].shape[0]
        stage_counts.loc[opponent, stage] = count

# Plotting
stage_counts.plot(kind='bar', figsize=(15, 8), width=0.8, color=plt.cm.viridis(np.linspace(0, 1, len(stages))))
plt.xlabel('Opponent Netplay Code')
plt.ylabel('Number of Games')
plt.title('Number of Games Against Top 10 MARTH Players on Each Stage (Mango as FALCO)')
plt.legend(title='Stage Name', labels=stages)

plt.show()

In [ ]:
# Filter games where Mango played as FALCO and opponent as FOX
falco_vs_fox_games = fox_games[fox_games['opponent_character_name'] == 'MARTH']

# Normalize the opponent_netplay_code
falco_vs_fox_games['normalized_opponent'] = falco_vs_fox_games['opponent_netplay_code'].apply(lambda x: x.split('#')[0])

# Find the top 5 normalized opponent_netplay_code
top_opponents = falco_vs_fox_games['normalized_opponent'].value_counts().head(10).index

# Prepare the data for plotting
stage_counts = pd.DataFrame(index=top_opponents, columns=stages)

# Populate the DataFrame with the number of games for each normalized opponent and stage
for opponent in top_opponents:
    for stage in stages:
        count = falco_vs_fox_games[
            (falco_vs_fox_games['normalized_opponent'] == opponent) & 
            (falco_vs_fox_games['stage_name'] == stage)
        ].shape[0]
        stage_counts.loc[opponent, stage] = count

# Plotting
stage_counts.plot(kind='bar', figsize=(15, 8), width=0.8, color=plt.cm.viridis(np.linspace(0, 1, len(stages))))
plt.xlabel('Opponent Netplay Code')
plt.ylabel('Number of Games')
plt.title('Number of Games Against Top 10 MARTH Players on Each Stage (Mango as FOX)')
plt.legend(title='Stage Name', labels=stages)

plt.show()

<h2> Explore Mango's Winrate </h2>

In [ ]:
from prettytable import PrettyTable

# Create the table
table1 = PrettyTable()
table1.field_names = ["Opponent Character", "Games Played", "Win Rate as Fox", "Win Rate as Falco"]

# Populate the table
for character in total_games.index:
    games_played = total_games[character]
    win_rate_fox = sorted_win_rates.loc['FOX', character]
    win_rate_falco = sorted_win_rates.loc['FALCO', character]
    table1.add_row([character, games_played, f"{win_rate_fox:.2f}", f"{win_rate_falco:.2f}"])

print("Mango's Win Rate Against Each Character")
print(table1)

print('mango plays better against Fox as Fox but he prefers Falco')

In [ ]:
table2 = PrettyTable()
table2.field_names = ["Stage Name", "Win Rate as Fox", "Win Rate as Falco"]

for stage in stages:
    win_rate_fox = df_mango_stats[(df_mango_stats['mango_character_name'] == 'FOX') & (df_mango_stats['stage_name'] == stage)]['mango_win'].mean()
    win_rate_falco = df_mango_stats[(df_mango_stats['mango_character_name'] == 'FALCO') & (df_mango_stats['stage_name'] == stage)]['mango_win'].mean()
    table2.add_row([stage, f"{win_rate_fox:.2f}", f"{win_rate_falco:.2f}"])

print("\nMango's Win Rate on Each Stage")
print(table2)


-mango's winrate over time

- Mango's winrate over time.
